# Stock Trade Information

In [22]:
import sys
import importlib
import pandas as pd

sys.path.append('..')
oxygene = importlib.import_module('oxygene')
tools = importlib.import_module('tools')

from joblib import Parallel, delayed

## Get the stock trade for the day

In [3]:
# benchmark: 25min
start = '20050101'
codes = list(map(tools.format_code, oxygene.AkShare.stock_quote(code_only=True)))
stock_data = Parallel(n_jobs=-1, backend='threading')(delayed(oxygene.AkShare.market_daily)(code, start=start) for code in codes)
data = pd.concat(stock_data, axis=0, keys=codes, names=['instrument', 'datetime'], sort=True)
data = data.swaplevel().sort_index()
data.to_parquet('../data/raw_data/em_daily_stock/daily_stock.parquet', compression='gzip')

After fetching data and store in `data` variable in memory, we need to dump it into the disk

In [24]:
# benchmark: 44s
oxygene.Dumper(
    data = data,
    date_col = None,
    inst_col = None,
    uri = '../data/em_qlib_day',
    mode = 'w',
    freq = 'day',
).dump()